## 빅 데이터 분석 및 시각화 개론 프로젝트
#주제: 대중교통 승하차 데이터를 이용한 우이신설선 수요 예측
## 18조: 장민우, 표성준, 한승규
---

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
subwayToBus = pd.read_csv('subwayToBus.csv')
subwayToBus.head()

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
0,1958,310,대화,3,예비군훈련장,36262,177627,464279
1,1958,310,대화,3,대화역,36281,177715,464099
2,1958,310,대화,3,대화역,36601,177718,464073
3,1805,150,송내,1,송내역,46039,178169,443269
4,1805,150,송내,1,송내역,46731,178200,443181


In [3]:
transfer_region_list = list()
subway_list = ['성신여대입구','길음','한성대입구','안암','보문','창신','제기동','신설동','동묘앞']
for station in subway_list:
    transfer_region_list.append(subwayToBus.where(subwayToBus['전철역명'] == station).dropna())
transfer_region_list[0]

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
1088,418.0,418,성신여대입구,4,삼선동주민센터,8176.0,201300.0,454575.0
1089,418.0,418,성신여대입구,4,성신여대입구,8269.0,201435.0,454875.0
1090,418.0,418,성신여대입구,4,성신여대입구,8270.0,201449.0,454884.0
1091,418.0,418,성신여대입구,4,돈암시장입구,8305.0,201461.0,454719.0
1092,418.0,418,성신여대입구,4,돈암시장입구,8306.0,201478.0,454709.0
1093,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8008.0,201576.0,454865.0
1094,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8007.0,201602.0,454913.0


In [4]:
transfer_region = list()
for i in transfer_region_list:
    temp_list = i['정류장ID'].tolist()
    for j in temp_list:
        transfer_region.append(int(j))

In [5]:
HomeRegionData = pd.read_csv('HomeRegion.csv')
HomeRegionData.head()

,지하철,정류장,코드
0,북한산우이,우이동,9500
1,북한산우이,우이동차고지종점,9294
2,북한산우이,우이동차고지기점,9289
3,북한산우이,우이동도선사입구,9102
4,북한산우이,우이동도선사입구,9101


In [6]:
home_region = list()
temp_list = HomeRegionData['코드'].tolist()
for j in temp_list:
    home_region.append(int(j))

In [7]:
busStationBoarding = pd.read_csv('BUS_STATION_BOARDING_MONTH_201710.csv')
busStationBoarding = busStationBoarding.where(busStationBoarding['사용일자'] == 20171001).dropna()
busStationBoarding.head()

,사용일자,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20171001.0,141,141번(도봉산~염곡동),109000031.0,10114,도봉소방서,106.0,73.0,20171004.0
1,20171001.0,141,141번(도봉산~염곡동),109000008.0,10008,신도봉사거리,26.0,155.0,20171004.0
2,20171001.0,141,141번(도봉산~염곡동),122000235.0,23339,하나은행사거리,114.0,15.0,20171004.0
3,20171001.0,141,141번(도봉산~염곡동),109000030.0,10113,도봉소방서,58.0,257.0,20171004.0
4,20171001.0,141,141번(도봉산~염곡동),122000192.0,23296,대림역삼아파트,174.0,33.0,20171004.0


In [8]:
def is_passing_by(region, boarding_df):
    result = boarding_df.where(boarding_df['버스정류장ARS번호'] == '0').dropna()
    for ID in region:
        if ID < 10000:
            IDstr = str(ID).zfill(5)
        else:
            IDstr = str(ID)        
        temp = boarding_df.where(boarding_df['버스정류장ARS번호'] == IDstr).dropna()
        result = result.append(temp,ignore_index=True)

    return result

transfer_region_bus = is_passing_by(transfer_region,busStationBoarding)
transfer_region_bus = transfer_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [9]:
home_region_bus = is_passing_by(home_region, busStationBoarding)
home_region_bus = home_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [13]:
common_line = []
for line in transfer_region_bus:
    if line in home_region_bus:
        if not "N" in line:
            common_line.append(line)
        
print(common_line)

['152', '1162', '2115', '1014', '162', '성북22', '성북20', '143', '104', '171', '109', '151', '1113', '1164', '1114', '110B', '7211', '1213', '153', '110A', '121', '1115', '1128', '101', '144']
